## Grabcat - Foreground/background segmentation of cats in images

In [2]:
import numpy as np
import math
import cv2 as cv
import glob
import time
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import scipy.io
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from sklearn.metrics import precision_score,recall_score,accuracy_score

In [3]:
def show_images(image_1, image_2=None, image_3=None, name_1='image_1', name_2='image_2', name_3='image_3', time_out=0):
    """
    Show images received as parameters.
    """
    cv.imshow(name_1, image_1)
    if image_2 is not None:
        cv.imshow(name_2, image_2)
    if image_3 is not None:
        cv.imshow(name_3, image_3)
    cv.waitKey(time_out)
    cv.destroyAllWindows()

In [4]:
def measure_performance(result,gt):
    """
    result : segmentation obtained
    gt: ground-truth segmentation 
    returns precision and recall
    """
    result_converted=np.asarray(result)
    result_converted[result_converted<200]=0
    result_converted[result_converted>200]=1
    result_converted=result_converted.flatten()
    gt_converted=np.asarray(gt)
    gt_converted[gt_converted<200]=0
    gt_converted[gt_converted>200]=1
    gt_converted=gt_converted.flatten()
    return precision_score(gt_converted,result_converted),recall_score(gt_converted,result_converted)

In [5]:
def get_regions(image):
    """
    image: image for segmentation
    -select the ROI
    -create list with background pixels
    -create list with foreground pixels
    returns the coordinates of region of interest, cropped_image, background and foreground pixels
    """
    r = cv.selectROI("select the area", image)
    cropped_image = image[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
    show_images(cropped_image)
    background=[]
    #left
    for i in range(image.shape[0]):
        for j in range(0,r[0]):
            background.append(image[i,j,:])
    #right
    for i in range(image.shape[0]):
        for j in range(r[2]+r[0],image.shape[1]):

            background.append(image[i,j,:])
    #up
    for i in range(0,r[1]):
        for j in range(r[0],r[0]+r[2]):

            background.append(image[i,j,:])    
    #down
    for i in range(r[1]+r[3],image.shape[0]):
        for j in range(r[0],r[0]+r[2]):
            background.append(image[i,j,:])  
    b=np.array(background)
    fb=cropped_image.reshape((-1, 3))
    return r,b,fb,cropped_image

In [6]:
def get_kmean_centroids(data,k):
    """
    data: data to be clustered
    k: number of clusters
    returns the center of clusters
    """
    kmeans = KMeans(init="random", n_clusters=k, n_init=10, max_iter=300, random_state=42)
    kmeans.fit(np.float64(data))
    return kmeans.cluster_centers_

In [7]:
def segment_image(cropped_image,iterations,centroids_b,centroids_f,b_orig,fb_orig,img_name):
    """
    cropped_image: region to be segmented
    iterations: number of iterations
    centroids_b: number of centroids for background
    centroids_f: number of centroids for foreground
    b_orig: original pixels for background
    fb_orig: original pixels from foreground and background region
    img_name: image name for segmented image
    -cluster background pixels
    -cluster pixels from foreground and background region
    -compute euclidean distance between pixels from cropped image and cluster centers
    -assign pixels to background if minimum distance is found between a cluster from backgound
    """
    b=b_orig.copy()
    fb=fb_orig.copy()
    for cnt in range(iterations):
        kmeans_b=get_kmean_centroids(b,centroids_b)
        kmeans_fb=get_kmean_centroids(fb,centroids_f)
        img_mask=np.zeros((cropped_image.shape[0],cropped_image.shape[1]))
        img_mask.fill(255)
        b=b_orig.copy()
        fb=fb_orig.copy()
        for i in range(cropped_image.shape[0]):
            for j in range(cropped_image.shape[1]):
                distances_b=[]
                distances_fb=[]
                for c in kmeans_b:
                    distances_b.append(math.dist(cropped_image[i,j,:], c))
                for c in kmeans_fb:
                    distances_fb.append(math.dist(cropped_image[i,j,:], c))
                min_b=min(distances_b)
                min_fb=min(distances_fb)
                if min_b<min_fb:
                    b=np.append(b,[cropped_image[i,j,:]],axis=0)
                    fb=np.delete(fb, np.where(fb == [cropped_image[i,j,:]])[0][0],axis=0)
                    img_mask[i,j]=0
        #show_images(img_mask)
        cv.imwrite("segmentations/"+img_name+'_'+str(cnt)+'.jpg', img_mask)
    return img_mask

In [22]:
image_path='07.jpg'
gt_image_path='07.png'
gt_image=cv.imread('ground_truth/'+gt_image_path,cv.IMREAD_GRAYSCALE)
img_name=image_path.split('.')[0]
image = cv.imread('images/'+image_path)
r,b_orig,fb_orig,cropped_image=get_regions(image)
mask=segment_image(cropped_image,5,10,7,b_orig,fb_orig,img_name)
full_image=np.zeros((image.shape[0],image.shape[1],1))
full_image[r[1]:r[1]+r[3],r[0]:r[0]+r[2]]=1
full_image[r[1]:r[1]+r[3],r[0]:r[0]+r[2]]=full_image[r[1]:r[1]+r[3],r[0]:r[0]+r[2]] * mask[:, :, np.newaxis]
show_images(full_image)
cv.imwrite("segmentations/"+img_name+'_classic'+'.jpg', full_image)
print(measure_performance(full_image.copy(),gt_image.copy()))

In [19]:
# https://www.docs.opencv.org/master/d8/d83/tutorial_py_grabcut.html
def run_grabcut(img):
    mask = np.zeros(img.shape[:2], np.uint8)
    bgdModel = np.zeros((1,65), np.float64)
    fgdModel = np.zeros((1,65), np.float64)
    rect = cv.selectROI(img)
    cv.destroyAllWindows()
    cv.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv.GC_INIT_WITH_RECT)
    
    mask2 = np.where((mask == 2)|(mask == 0),0,255).astype('uint8')
    return mask2
image_path='07.jpg'    
img = cv.imread('images/07.jpg')
img_name=image_path.split('.')[0]
gt_image_path='07.png'
gt_image=cv.imread('ground_truth/'+gt_image_path,cv.IMREAD_GRAYSCALE)
mask = run_grabcut(img)
show_images(image_1=mask, name_1='img_grabcut') 
cv.imwrite("segmentations/"+img_name+'_grabcut'+'.jpg', mask)
print(measure_performance(mask,gt_image))

In [8]:
NUMBER_DIFFERENT_COLORS = 50
random_colors = np.uint8(np.random.randint(255, size=(NUMBER_DIFFERENT_COLORS, 3)))

def label_to_rgb(img_label, colors=random_colors):
    """
    It colors the image based on the label number.
    :param img_label. The 'image' containing a label for each patch/portion of the image.
    :param colors
    :return the colored image.
    """
    h, w = img_label.shape
    fake_colored_label_image = np.uint8(np.zeros((h, w, 3))) 
    
    number_labels = img_label.max() + 1
    number_different_colors = colors.shape[0] 
    
    for label in range(number_labels):
        rows, columns = np.where(img_label == label)
        fake_colored_label_image[rows, columns, :] = colors[label % number_different_colors]
    return fake_colored_label_image

In [9]:
def apply_slic(image, show_images_=False):
    """
    Apply the SLIC algorithm on the image received as paramters.
    :param image.
    :show_images_. If True the result of the algorithm is shown.
    :return The superpixels obtained by running the SLIC algorithm.
    """
    segments_slic = slic(image, n_segments=250, compactness=10, start_label=1)  
    if show_images_:
        colored_segments = label_to_rgb(segments_slic)
        show_images(image_1=image, image_2=colored_segments, name_1='image', name_2='splic_segmentation',
                    image_3=mark_boundaries(image, segments_slic), name_3='boundaries')
    return segments_slic

In [10]:
def get_regions_super(image,S):
    """
    image: image for segmentation
    -select the ROI
    -create list with background pixels
    -create list with foreground pixels
    returns the coordinates of region of interest, cropped_image, background and foreground pixels
    """
    r = cv.selectROI("select the area", image)
    cropped_image = image[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
    
    show_images(cropped_image)
    
    cropped_image_super=S[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
    
    number_labels = S.max() + 1
    fb=[]
    b=[]
    for i in range(1, number_labels):
        if i in  cropped_image_super:
            fb.append(i)
    for i in range(1, number_labels):
        if i not in fb:
            b.append(i)
    return r,b,fb,cropped_image
   

In [11]:
def get_mean_color_superpixel_response(image,S, label):    
    rows, columns = np.where(S == label )
    superpixel_colors = image[rows, columns,:]
    superpixel_mean_color= np.mean(superpixel_colors, axis=0)         
    return superpixel_mean_color

In [12]:
def segment_image_super_pixels(S,iterations,centroids_b,centroids_f,b_orig,fb_orig,img_name):
    """
    S: superpixels image
    iterations: number of iterations
    centroids_b: number of centroids for background
    centroids_f: number of centroids for foreground
    b_orig: original superpixels for background
    fb_orig: original superpixels from foreground and background region
    img_name: image name for segmented image
    -cluster background pixels
    -cluster pixels from foreground and background region
    -compute euclidean distance between pixels from cropped image and cluster centers
    -assign pixels to background if minimum distance is found between a cluster from background
    """
    features_fb_orig=[]
    for x in fb_orig:
        features_fb_orig.append(get_mean_color_superpixel_response(image,S, x))
    b=b_orig.copy()
    fb=fb_orig.copy()
    for cnt in range(iterations):
        features_b=[]
        features_fb=[]
        for x in b:
            features_b.append(get_mean_color_superpixel_response(image,S, x))
        for x in fb:
            features_fb.append(get_mean_color_superpixel_response(image,S, x))
        h, w = S.shape
        rez_mask = np.uint8(np.zeros((h, w, 1)))     
        rez_mask.fill(0)
        kmeans_b=get_kmean_centroids(features_b,centroids_b)
        kmeans_fb=get_kmean_centroids(features_fb,centroids_f)
        b=b_orig.copy()
        fb=fb_orig.copy()
        for i in range(0,len(fb_orig)):
                distances_b=[]
                distances_fb=[]
                for c in kmeans_b:
                    distances_b.append(math.dist(features_fb_orig[i], c))
                for c in kmeans_fb:
                    distances_fb.append(math.dist(features_fb_orig[i], c))
                min_b=min(distances_b)
                min_fb=min(distances_fb)
                if min_b<min_fb:
                    b.append(fb_orig[i])
                    fb.remove(fb_orig[i])
        for label in fb:
            rows, columns = np.where(S == label)
            rez_mask[rows, columns, :] = 255
        cv.imwrite("segmentations/"+img_name+'_super_'+str(cnt)+'.jpg', rez_mask)
    return rez_mask

In [13]:
image_path='05.jpg'
gt_image_path='05.png'
gt_image=cv.imread('ground_truth/'+gt_image_path,cv.IMREAD_GRAYSCALE)
img_name=image_path.split('.')[0]
image = cv.imread('images/'+image_path)
S=apply_slic(image, show_images_=False)
r,b,fb,cropped_image=get_regions_super(image,S)
mask=segment_image_super_pixels(S,50,5,5,b,fb,img_name)
print(measure_performance(mask,gt_image))